# Nettoyage des données de Formula 1

## 1. Objectifs du Notebook
Ce notebook a pour objectif principal de préparer les données de Formula 1 en vue d’analyses approfondies. Le dataset couvre une période allant de 1950 à 2024, incluant divers aspects tels que les circuits, les pilotes, les classements, les résultats, et bien plus. Pour garantir la fiabilité des analyses, plusieurs tâches de nettoyage des données seront effectuées :

#### Validation des types de données :

- Vérifier que chaque colonne utilise un type de données approprié (numérique, catégoriel, etc.).


#### Identification et gestion des valeurs manquantes :

- Repérer les colonnes ou lignes contenant des données manquantes et appliquer des stratégies adaptées (suppression ou imputation).
  
#### Traitement des doublons :

- Vérifier l’existence de doublons dans les fichiers et les éliminer pour éviter les biais dans les analyses.

#### Gestion des incohérences :

- Identifier et corriger les éventuelles incohérences dans les données (par exemple, une valeur qui sort du bon intervalle).

Le but final est de produire un jeu de données propre, structuré et prêt pour des analyses exploratoires ou la construction de modèles de machine learning.

## 2. Importation des packages

In [2]:
import pandas as pd

## 3. Chargement des données

Pour analyser l'impact de la météo sur les performances des pilotes en Formule 1, plusieurs fichiers de données sont utilisés, chacun apportant des informations complémentaires :

- Weather(2018 - 2024).csv : Contient les données météorologiques pour chaque course, incluant des variables telles que la température, les précipitations et les conditions générales (pluie, soleil, etc.). C’est le fichier central pour l’analyse météo.
- races.csv : Fournit des informations contextuelles sur chaque course (lieu, date, etc.), nécessaires pour associer les données météo aux performances des pilotes.
- drivers.csv : Liste les pilotes, permettant d’identifier et d’évaluer leurs performances individuelles.
- results.csv : Contient les résultats détaillés des courses, notamment les positions finales, les points, les meilleurs tours, et les abandons, qui serviront à mesurer les impacts de la météo.
- lap_times.csv : Fournit des données détaillées sur les temps au tour, utiles pour analyser comment les conditions météo influencent la régularité et la vitesse.
- status.csv : Donne des informations sur les statuts des pilotes à la fin des courses (abandon, accident, etc.), souvent affectés par la météo.

In [23]:
# Chargement des données 
meteo = pd.read_csv('Data/Weather(2018 - 2024).csv')
courses = pd.read_csv('Data/races.csv')
pilotes = pd.read_csv('Data/drivers.csv')
resultat = pd.read_csv('Data/results.csv')
laps_times = pd.read_csv('Data/lap_times.csv')
status = pd.read_csv('Data/status.csv')

# Afficher les dimensions de chaque dataset
print("Dimensions de 'meteo' :", meteo.shape)
print("Dimensions de 'courses' :", courses.shape)
print("Dimensions de 'pilotes' :", pilotes.shape)
print("Dimensions de 'resultats' :", resultat.shape)
print("Dimensions de 'laps_times' :", laps_times.shape)
print("Dimensions de 'status' :", status.shape)

Dimensions de 'meteo' : (21859, 10)
Dimensions de 'courses' : (1125, 18)
Dimensions de 'pilotes' : (859, 9)
Dimensions de 'resultats' : (26519, 18)
Dimensions de 'laps_times' : (575029, 6)
Dimensions de 'status' : (139, 2)


## 4. Nettoyage de la base de données Météo

In [3]:
## Affichages des 5 premières lignes
meteo.tail(5)

,Time,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed,Round Number,Year
21854,0 days 02:23:57.512000,25.9,60.0,1018.0,False,29.3,122,2.6,24,2024
21855,0 days 02:24:57.504000,25.9,60.0,1018.0,False,29.4,84,2.4,24,2024
21856,0 days 02:25:57.504000,25.8,60.0,1018.0,False,29.4,34,1.2,24,2024
21857,0 days 02:26:57.518000,25.8,60.0,1018.0,False,29.1,116,1.2,24,2024
21858,0 days 02:27:57.568000,25.8,60.0,1018.0,False,29.1,116,1.2,24,2024


In [4]:
## Vérification des types des données
meteo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21859 entries, 0 to 21858
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Time           21859 non-null  object 
 1   AirTemp        21859 non-null  float64
 2   Humidity       21859 non-null  float64
 3   Pressure       21859 non-null  float64
 4   Rainfall       21859 non-null  bool   
 5   TrackTemp      21859 non-null  float64
 6   WindDirection  21859 non-null  int64  
 7   WindSpeed      21859 non-null  float64
 8   Round Number   21859 non-null  int64  
 9   Year           21859 non-null  int64  
dtypes: bool(1), float64(5), int64(3), object(1)
memory usage: 1.5+ MB


### Vérification des données manquantes

In [5]:
# Fonction qui vérifie s'il y a des valeurs manquantes dans un dataset
def check_missing_values(df, dataset_name="Dataset"):
    if df.isnull().sum().sum() == 0 :
        print(f'La base de données {dataset_name} ne contient pas des valeurs manquantes')
    else:
        print(f'La base de données {dataset_name} contient {df.isnull().sum().sum()} valeur(s) manquante(s)')

check_missing_values(meteo, 'Météo')

La base de données Météo ne contient pas des valeurs manquantes


### Vérification des doublons

In [6]:
# Fonction qui vérifie s'il y a des doublons dans un dataset
def check_duplicates(df, dataset_name="Dataset"):
    duplicates = df.duplicated().sum()  # Compter le nombre de doublons
    if duplicates > 0:
        print(f"{dataset_name} contient {duplicates} doublon(s).")
    else:
        print(f"{dataset_name} ne contient pas de doublons.")

check_duplicates(meteo, 'Météo')

Météo contient 481 doublon(s).


In [7]:
# Suppression des doublons
def remove_duplicates(df):
    return df.drop_duplicates()

meteo_cleaned = remove_duplicates(meteo)

In [8]:
# Vérification des la suppression des doublons
check_duplicates(meteo_cleaned, 'Météo')

Météo ne contient pas de doublons.


### Vérification des intervalles pour les colonnes numériques

In [9]:
meteo_cleaned.describe()

,AirTemp,Humidity,Pressure,TrackTemp,WindDirection,WindSpeed,Round Number,Year
count,21378.000000,21378.000000,21378.000000,21378.000000,21378.000000,21378.000000,21378.000000,21378.000000
mean,23.471181,54.709664,986.793068,35.224815,173.404107,1.637702,11.524652,2021.365656
std,4.988278,17.379909,50.957704,9.251853,104.043864,1.148441,6.302429,1.964110
min,8.900000,5.000000,778.500000,13.800000,0.000000,0.000000,1.000000,2018.000000
25%,20.000000,42.000000,986.200000,29.000000,95.000000,0.800000,6.000000,2020.000000
50%,23.200000,55.000000,1006.600000,34.800000,170.000000,1.400000,12.000000,2022.000000
75%,27.300000,65.775000,1013.700000,42.200000,264.000000,2.200000,17.000000,2023.000000
max,37.200000,97.500000,1023.500000,67.000000,359.000000,10.100000,24.000000,2024.000000


Toutes les variables du dataset sont comprises dans des intervalles réalistes, à l'exception de la variable Pressure. 

Cette dernière présente des valeurs comprises entre 778 et 1023, ce qui est incohérent avec les observations météorologiques réelles. En effet, la pression atmosphérique la plus faible jamais enregistrée est de 879 hPa (source : https://alarmemeteo.ch/blog/la-pression-atmospherique-la-declencheuse-des-vents-et-des-tempetes.html). 

Par conséquent, dans la suite de ce projet, seules les pressions atmosphériques égales ou supérieures à 900 hPa seront conservées pour garantir une analyse cohérente et fiable.

In [10]:
# Proportion des meteo qui ont une pression alterieur inferieur à 950
(meteo['Pressure'] < 900).sum() / len(meteo) * 100

4.336886408344389

Nous constatons que Environ 4% des courses ont une pression atmospherique inférieur à 900 hPa

In [11]:
# Suppression des valeurs de pression atmosphérique < 950
meteo_cleaned = meteo_cleaned[meteo_cleaned['Pressure'] >= 950]

# Afficher les 5 premières lignes pour vérifier
meteo_cleaned.describe()

,AirTemp,Humidity,Pressure,TrackTemp,WindDirection,WindSpeed,Round Number,Year
count,18220.000000,18220.000000,18220.000000,18220.000000,18220.000000,18220.000000,18220.000000,18220.000000
mean,23.661465,55.770692,1003.965368,34.556811,171.775521,1.672849,10.689133,2021.338749
std,5.085639,17.062601,14.715938,8.795537,105.520129,1.186057,6.047923,1.952609
min,9.200000,5.000000,959.900000,13.800000,0.000000,0.000000,1.000000,2018.000000
25%,19.700000,44.000000,995.900000,28.700000,93.000000,0.800000,5.000000,2020.000000
50%,24.000000,56.000000,1008.500000,34.000000,167.000000,1.400000,11.000000,2022.000000
75%,27.700000,66.200000,1014.900000,41.000000,263.000000,2.200000,15.000000,2023.000000
max,37.200000,97.500000,1023.500000,67.000000,359.000000,10.100000,24.000000,2024.000000


### Enregistrement des données nettoyés dans un fichier csv

In [10]:
def dataset_to_csv(df, name):
    df.to_csv(f'Cleaned_Data/{name}.csv', index=False)

    print(f"Les données nettoyées ont été enregistrées dans le fichier 'Cleaned_Data/{name}.csv'.")

#dataset_to_csv(meteo_cleaned, 'meteo_cleaned')

## 5. Nettoyage de la base de données Courses

### 5.1 Vérification des types de données

In [13]:
# Affichage des 5 premières lignes
courses.head(5)

,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N


In [14]:
# Affichage des types de données
courses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1125 entries, 0 to 1124
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   raceId       1125 non-null   int64 
 1   year         1125 non-null   int64 
 2   round        1125 non-null   int64 
 3   circuitId    1125 non-null   int64 
 4   name         1125 non-null   object
 5   date         1125 non-null   object
 6   time         1125 non-null   object
 7   url          1125 non-null   object
 8   fp1_date     1125 non-null   object
 9   fp1_time     1125 non-null   object
 10  fp2_date     1125 non-null   object
 11  fp2_time     1125 non-null   object
 12  fp3_date     1125 non-null   object
 13  fp3_time     1125 non-null   object
 14  quali_date   1125 non-null   object
 15  quali_time   1125 non-null   object
 16  sprint_date  1125 non-null   object
 17  sprint_time  1125 non-null   object
dtypes: int64(4), object(14)
memory usage: 158.3+ KB


### Échantillonage des courses du 1950-2024 à 2018-2024

* Les données météorologiques que nous avons à notre disposition sont de 2018 à 2024. Ce qui nous empêche de pouvoir mener une analyse météorologique avant 2018. 

* Par conséquent, Dans la suite de ce projet nous allons nous focaliser sur les années 2018-2024.

In [15]:
# Récuperer les courses de 2018 à 2024
courses_2018_2024 = courses[(courses['year'] >= 2018) & (courses['year'] <= 2024)]

# Reindexer le dataset
courses_2018_2024 = courses_2018_2024.reset_index(drop=True)

# Afficher les 5 premières lignes
courses_2018_2024.head()

,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,989,2018,1,1,Australian Grand Prix,2018-03-25,05:10:00,http://en.wikipedia.org/wiki/2018_Australian_G...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1,990,2018,2,3,Bahrain Grand Prix,2018-04-08,15:10:00,http://en.wikipedia.org/wiki/2018_Bahrain_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
2,991,2018,3,17,Chinese Grand Prix,2018-04-15,06:10:00,http://en.wikipedia.org/wiki/2018_Chinese_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
3,992,2018,4,73,Azerbaijan Grand Prix,2018-04-29,12:10:00,http://en.wikipedia.org/wiki/2018_Azerbaijan_G...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
4,993,2018,5,4,Spanish Grand Prix,2018-05-13,13:10:00,http://en.wikipedia.org/wiki/2018_Spanish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N


### Analyse du pertinance des variables

In [16]:
columns = ['fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time','quali_date', 'quali_time', 'sprint_date', 'sprint_time', 'time', 'date']

for column in columns: 
    print(column, " : ")
    print(courses_2018_2024[column].describe(), "\n")

fp1_date  : 
count     149
unique     91
top        \N
freq       59
Name: fp1_date, dtype: object 

fp1_time  : 
count     149
unique     21
top        \N
freq       81
Name: fp1_time, dtype: object 

fp2_date  : 
count     149
unique     91
top        \N
freq       59
Name: fp2_date, dtype: object 

fp2_time  : 
count     149
unique     20
top        \N
freq       81
Name: fp2_time, dtype: object 

fp3_date  : 
count     149
unique     73
top        \N
freq       77
Name: fp3_date, dtype: object 

fp3_time  : 
count     149
unique     19
top        \N
freq       96
Name: fp3_time, dtype: object 

quali_date  : 
count     149
unique     91
top        \N
freq       59
Name: quali_date, dtype: object 

quali_time  : 
count     149
unique     16
top        \N
freq       81
Name: quali_time, dtype: object 

sprint_date  : 
count     149
unique     19
top        \N
freq      131
Name: sprint_date, dtype: object 

sprint_time  : 
count     149
unique     13
top        \N
freq      134
Name:

- Les variables ['fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time'] contiennent plus de 39 % de valeurs manquantes (\N). De plus, ces variables se réfèrent principalement aux horaires des différentes sessions de la course (essais libres, qualifications, sprint, etc.), qui ne sont pas directement liées aux conditions météorologiques ayant un impact sur les performances des pilotes. Par conséquent, nous avons décidé de retirer ces variables pour simplifier l'analyse en nous concentrant sur les éléments les plus pertinents. La variable 'url' sera conservée, car elle peut être utile pour obtenir plus d'informations sur une course si nécessaire.

- En revanche, les variables 'date' et 'time' de la course restent pertinentes. Elles permettent de situer les données météorologiques dans le temps, ce qui est essentiel pour l'analyse des conditions climatiques et leur impact sur les performances des pilotes. Toutefois, la variable 'time' sera supprimée. En effet, dans la table des données météorologiques, nous disposons déjà d'informations météo associées à chaque minute de la course, ce qui rend la variable 'time' redondante dans le contexte de notre analyse.

- Les variables 'year', 'round', 'circuitId' et 'name' sont également conservées car elles permettent d'identifier chaque course, la saison et le circuit associés. Ces informations sont essentielles pour relier les données météorologiques aux événements spécifiques de la compétition et pour étudier l'impact des conditions climatiques sur les performances des pilotes en fonction de l'année, du circuit et du tour de la compétition.


In [17]:
# Suppression des colonnes non indispensable
courses_cleaned = courses_2018_2024[['raceId', 'year', 'round', 'circuitId', 'name', 'date', 'time']]
courses_cleaned.head()

,raceId,year,round,circuitId,name,date,time
0,989,2018,1,1,Australian Grand Prix,2018-03-25,05:10:00
1,990,2018,2,3,Bahrain Grand Prix,2018-04-08,15:10:00
2,991,2018,3,17,Chinese Grand Prix,2018-04-15,06:10:00
3,992,2018,4,73,Azerbaijan Grand Prix,2018-04-29,12:10:00
4,993,2018,5,4,Spanish Grand Prix,2018-05-13,13:10:00


### Analyse des valeurs manquantes
        

In [18]:
check_missing_values(courses_cleaned, 'Courses')

La base de données Courses ne contient pas des valeurs manquantes


### Analyse des doublons        

In [19]:
check_duplicates(courses_cleaned, 'Courses')

Courses ne contient pas de doublons.


### Enregistrement des données nettoyés dans un fichier csv

In [23]:
dataset_to_csv(courses_cleaned, 'courses_cleaned')
courses_cleaned.head()

,raceId,year,round,circuitId,name,date,time
0,989,2018,1,1,Australian Grand Prix,2018-03-25,05:10:00
1,990,2018,2,3,Bahrain Grand Prix,2018-04-08,15:10:00
2,991,2018,3,17,Chinese Grand Prix,2018-04-15,06:10:00
3,992,2018,4,73,Azerbaijan Grand Prix,2018-04-29,12:10:00
4,993,2018,5,4,Spanish Grand Prix,2018-05-13,13:10:00


## 5. Nettoyage de la base de données des pilotes

### Analyse des types de données

In [4]:
# Afficher les 5 premieres lignes
pilotes.head(5)

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen


In [6]:
# Vérification du type des données
pilotes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 859 entries, 0 to 858
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   driverId     859 non-null    int64 
 1   driverRef    859 non-null    object
 2   number       859 non-null    object
 3   code         859 non-null    object
 4   forename     859 non-null    object
 5   surname      859 non-null    object
 6   dob          859 non-null    object
 7   nationality  859 non-null    object
 8   url          859 non-null    object
dtypes: int64(1), object(8)
memory usage: 60.5+ KB


### Choix des variables pertinantes

Pour le fichier drivers.csv, seules les colonnes driverId, code, forename, surname et url seront conservées :

- Les colonnes driverId, forename et surname permettent d'identifier clairement les pilotes, ce qui est essentiel pour relier leurs performances aux données météorologiques.
- La colonne url est également utile pour obtenir des informations supplémentaires sur un pilote, si nécessaire.

Les autres colonnes ont été jugées non pertinentes pour analyser l'impact des conditions météorologiques sur les performances des pilotes :

- code s'agit généralement d'un code à trois lettres pour identifier les pilotes, principalement utilisé dans les affichages et classements, mais sans impact direct pour analyser les effets de la météo.
- driverRef est une simple référence textuelle.
- number est utilisé pour le classement ou l'affichage des pilotes lors des courses, mais n'apporte pas d'information sur leurs performances liées à la météo.
- dob (date de naissance), bien qu'elle permette de calculer l'âge, n'est pas directement corrélée avec l'impact des conditions météorologiques dans ce contexte d'analyse.
- nationality, bien qu'intéressante sur le plan descriptif, n'a pas d'influence mesurable sur les performances des pilotes en fonction des conditions climatiques.

Ces colonnes seront donc supprimées pour simplifier l'analyse et se concentrer sur les variables directement utiles.

In [21]:
pilotes_cleaned = pilotes[['driverId','forename','surname','url']]

In [22]:
pilotes_cleaned.head()

,driverId,forename,surname,url
0,1,Lewis,Hamilton,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,Nick,Heidfeld,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,Nico,Rosberg,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,Fernando,Alonso,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,Heikki,Kovalainen,http://en.wikipedia.org/wiki/Heikki_Kovalainen


### Analyse des valeurs manquantes

In [25]:
check_missing_values(pilotes_cleaned, 'pilotes_cleaned')

La base de données pilotes_cleaned ne contient pas des valeurs manquantes


### Analyse des doublons

In [26]:
check_duplicates(pilotes_cleaned, 'pilotes_cleaned')

pilotes_cleaned ne contient pas de doublons.


### Enregistrement des données nettoyés dans un fichier csv

In [28]:
dataset_to_csv(pilotes_cleaned, 'pilotes_cleaned')

Les données nettoyées ont été enregistrées dans le fichier 'Cleaned_Data/pilotes_cleaned.csv'.


## 6. Nettoyae des données des resultats des courses

## Vérification des types de données

In [6]:
# Afficher les 5 premières ligne
resultat.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


In [7]:
# Description des types de données
resultat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26519 entries, 0 to 26518
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   resultId         26519 non-null  int64  
 1   raceId           26519 non-null  int64  
 2   driverId         26519 non-null  int64  
 3   constructorId    26519 non-null  int64  
 4   number           26519 non-null  object 
 5   grid             26519 non-null  int64  
 6   position         26519 non-null  object 
 7   positionText     26519 non-null  object 
 8   positionOrder    26519 non-null  int64  
 9   points           26519 non-null  float64
 10  laps             26519 non-null  int64  
 11  time             26519 non-null  object 
 12  milliseconds     26519 non-null  object 
 13  fastestLap       26519 non-null  object 
 14  rank             26519 non-null  object 
 15  fastestLapTime   26519 non-null  object 
 16  fastestLapSpeed  26519 non-null  object 
 17  statusId    

### Sélection des variables pertinantes

Pour analyser l’impact de la météo sur les performances des pilotes, seules les variables directement liées aux résultats et aux performances des courses seront retenues :

- resultId, raceId, driverId : Identifiants nécessaires pour relier les performances aux courses, pilotes, et conditions météo.
- positionOrder : Représente le classement final, indicateur clé de performance.
- points : Évalue le succès global des pilotes sous différentes conditions météorologiques.
- laps : Indique le nombre de tours complétés, utile pour détecter les abandons liés aux conditions.
- fastestLapTime, fastestLapSpeed : Mesures de performance maximale, permettant d'évaluer l'impact des conditions météo sur la vitesse.
- statusId : Permet de suivre les abandons ou incidents, souvent influencés par la météo.

Exclusion des autres variables : Les variables comme grid, milliseconds, ou rank sont redondantes ou moins pertinentes pour évaluer directement l’influence de la météo. Des colonnes purement techniques (ex. : number, constructorId) n’apportent pas de valeur analytique pour cet objectif spécifique.


In [ ]:
resultat_cleaned = resultat[['resultId', 'raceId', 'driverId', 'positionOrder', 'points', 'laps', 'fastestLapTime', 'fastestLapSpeed', 'statusId']]

resultat_cleaned.head()

### Analyses des valeurs manquantes

In [7]:
check_missing_values(resultat_cleaned, 'resultat_cleaned')

La base de données resultat_cleaned ne contient pas des valeurs manquantes


### Analyses des doublons

In [8]:
check_duplicates(resultat_cleaned, 'resultat_cleaned')

resultat_cleaned ne contient pas de doublons.


### Enregistrement des données nettoyés dans un fichier csv

In [11]:
dataset_to_csv(resultat_cleaned, 'resultat_cleaned')

Les données nettoyées ont été enregistrées dans le fichier 'Cleaned_Data/resultat_cleaned.csv'.


## 7. Nettoyage de la base de données laps_times

### Vérification des types de données

In [17]:
# Afficher les 5 premières lignes
laps_times.head()

,raceId,driverId,lap,position,time,milliseconds
575024,1131,858,65,19,1:10.742,70742
575025,1131,858,66,19,1:10.855,70855
575026,1131,858,67,19,1:12.454,72454
575027,1131,858,68,19,1:13.607,73607
575028,1131,858,69,19,1:11.489,71489


In [16]:
## Affiche les infomations sur chaque variable
laps_times.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 575029 entries, 0 to 575028
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   raceId        575029 non-null  int64 
 1   driverId      575029 non-null  int64 
 2   lap           575029 non-null  int64 
 3   position      575029 non-null  int64 
 4   time          575029 non-null  object
 5   milliseconds  575029 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 26.3+ MB


### Analyse des valeurs manquantes

In [18]:
check_missing_values(laps_times, 'laps_times')

La base de données laps_times ne contient pas des valeurs manquantes


### Analyse des doublons

In [19]:
check_duplicates(laps_times, 'laps_times')

laps_times ne contient pas de doublons.


### Enregistrement des données nettoyés dans un fichier csv

In [22]:
dataset_to_csv(laps_times, 'laps_times_cleaned')

Les données nettoyées ont été enregistrées dans le fichier 'Cleaned_Data/laps_times_cleaned.csv'.


## 8. Nettoyage de la base de données status

### Vérification des types de données

In [26]:
# Afficher les premières lignes
status.head()

,statusId,status
0,1,Finished
1,2,Disqualified
2,3,Accident
3,4,Collision
4,5,Engine


In [27]:
## Afficher les informations sur les variables
status.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   statusId  139 non-null    int64 
 1   status    139 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.3+ KB


### Analyse des valeurs manquantes

In [29]:
check_missing_values(status, 'status')

La base de données status ne contient pas des valeurs manquantes


### Analyse des doublons

In [30]:
check_duplicates(status, 'status')

status ne contient pas de doublons.


### Affichage des differents status

In [40]:
print(status['status'].unique().shape[0])
print(status['status'].unique())

139
['Finished' 'Disqualified' 'Accident' 'Collision' 'Engine' 'Gearbox'
 'Transmission' 'Clutch' 'Hydraulics' 'Electrical' '+1 Lap' '+2 Laps'
 '+3 Laps' '+4 Laps' '+5 Laps' '+6 Laps' '+7 Laps' '+8 Laps' '+9 Laps'
 'Spun off' 'Radiator' 'Suspension' 'Brakes' 'Differential' 'Overheating'
 'Mechanical' 'Tyre' 'Driver Seat' 'Puncture' 'Driveshaft' 'Retired'
 'Fuel pressure' 'Front wing' 'Water pressure' 'Refuelling' 'Wheel'
 'Throttle' 'Steering' 'Technical' 'Electronics' 'Broken wing'
 'Heat shield fire' 'Exhaust' 'Oil leak' '+11 Laps' 'Wheel rim'
 'Water leak' 'Fuel pump' 'Track rod' '+17 Laps' 'Oil pressure' '+42 Laps'
 '+13 Laps' 'Withdrew' '+12 Laps' 'Engine fire' 'Engine misfire'
 '+26 Laps' 'Tyre puncture' 'Out of fuel' 'Wheel nut' 'Not classified'
 'Pneumatics' 'Handling' 'Rear wing' 'Fire' 'Wheel bearing' 'Physical'
 'Fuel system' 'Oil line' 'Fuel rig' 'Launch control' 'Injured' 'Fuel'
 'Power loss' 'Vibrations' '107% Rule' 'Safety' 'Drivetrain' 'Ignition'
 'Did not qualify' 'Inj

### Enregistrement des données nettoyés dans un fichier csv

In [42]:
dataset_to_csv(status, 'status_cleaned')

Les données nettoyées ont été enregistrées dans le fichier 'Cleaned_Data/status_cleaned.csv'.
